In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!ls '/content/gdrive/MyDrive/'

 A3_files			    MakeupMid.ipynb
 cleaned_data.csv		   'Masters Capstone Project'
'Colab Notebooks'		    MS_Capstone_data
'Copy of Deepti_Baragi2.ipynb'	   'Recommender Systems - Project ppt.gslides'
 CV.gdoc			    rs_proj
 CV.pdf				    runs
 data_split.ipynb		   'Social Media Mining Assign3'
 Deepti_Baragi			    TC4
 Deepti_Baragi_CV.pdf		   'Template - Cyberdome Intern Questionnaire.docx'
'Deepti_Baragi – Homework 1.gdoc'   trail1.ipynb
'Deepti shout-out.mov'		    trial-1
 etext-all			   'Untitled document.gdoc'
'Fake Company Employees.gsheet'    'Untitled presentation (1).gslides'
'Fake Company.gsheet'		   'Untitled presentation.gslides'
'Flash cards.gslides'		    yelp_academic_dataset_review.json
 ImageSegmentationYOLOv8


In [ ]:
DATA_DIR = '/content/gdrive/MyDrive/ImageSegmentationYOLOv8/Image_Seg_Unet'

In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

# U-Net architecture
def unet(input_shape):
    inputs = tf.keras.Input(shape=input_shape)

    # Contracting Path
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = layers.Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = layers.Conv2D(512, 3, activation='relu', padding='same')(conv4)
    drop4 = layers.Dropout(0.5)(conv4)
    pool4 = layers.MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = layers.Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = layers.Conv2D(1024, 3, activation='relu', padding='same')(conv5)
    drop5 = layers.Dropout(0.5)(conv5)

    # Expansive Path
    up6 = layers.Conv2D(512, 2, activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(drop5))
    merge6 = layers.concatenate([drop4, up6], axis=3)
    conv6 = layers.Conv2D(512, 3, activation='relu', padding='same')(merge6)
    conv6 = layers.Conv2D(512, 3, activation='relu', padding='same')(conv6)

    up7 = layers.Conv2D(256, 2, activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(conv6))
    merge7 = layers.concatenate([conv3, up7], axis=3)
    conv7 = layers.Conv2D(256, 3, activation='relu', padding='same')(merge7)
    conv7 = layers.Conv2D(256, 3, activation='relu', padding='same')(conv7)

    up8 = layers.Conv2D(128, 2, activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(conv7))
    merge8 = layers.concatenate([conv2, up8], axis=3)
    conv8 = layers.Conv2D(128, 3, activation='relu', padding='same')(merge8)
    conv8 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv8)

    up9 = layers.Conv2D(64, 2, activation='relu', padding='same')(layers.UpSampling2D(size=(2, 2))(conv8))
    merge9 = layers.concatenate([conv1, up9], axis=3)
    conv9 = layers.Conv2D(64, 3, activation='relu', padding='same')(merge9)
    conv9 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv9)

    outputs = layers.Conv2D(1, 1, activation='sigmoid')(conv9)

    model = models.Model(inputs=inputs, outputs=outputs)
    return model


def load_data(images_folder, labels_folder):
    images = []
    labels = []
    for image_filename in os.listdir(images_folder):
        print("Processing image:", image_filename)
        img = cv2.imread(os.path.join(images_folder, image_filename))
        if img is None:
            print("Error: Unable to read image:", image_filename)
            continue
        img = cv2.resize(img, (256, 256))
        img = img / 255.0  # Normalize pixel values to [0, 1]


        label_filename = image_filename.split('.')[0] + '.png'
        label = cv2.imread(os.path.join(labels_folder, label_filename), cv2.IMREAD_GRAYSCALE)
        if label is None:
            print("Error: Unable to read label for image:", image_filename)
            continue
        label = cv2.resize(label, (256, 256))  # Resize labels to match the images
        label = np.expand_dims(label, axis=-1)  # Add channel dimension for compatibility with model output
        label = label / 255.0  # Normalize pixel values to [0, 1]

        images.append(img)
        labels.append(label)

    return np.array(images), np.array(labels)

train_images_folder = '/content/gdrive/MyDrive/ImageSegmentationYOLOv8/Image_Seg_Unet/images/train/'
val_images_folder = '/content/gdrive/MyDrive/ImageSegmentationYOLOv8/Image_Seg_Unet/images/val/'
labels_folder = '/content/gdrive/MyDrive/ImageSegmentationYOLOv8/Image_Seg_Unet/Seg_Research_Photos_2023/'


all_images, all_labels = load_data(train_images_folder, labels_folder)


train_images, val_images, train_labels, val_labels = train_test_split(all_images, all_labels, test_size=0.2, random_state=42)


input_shape = (256, 256, 3)  # Assuming RGB images

# Create U-Net model
model = unet(input_shape)


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=10, batch_size=8, validation_data=(val_images, val_labels))

# Evaluate the model
loss, accuracy = model.evaluate(val_images, val_labels)
print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)

Processing image: 4th St SE from Chesapeake St to Livingston Ter (1).jpg
Processing image: 44TH ST NE FROM EDSON PL NE TO FOOTE ST NE.jpg
Processing image: 43RD ST NW FROM MURDOCK MILL RD NW TO BUTTERWORTH PL NW.jpg
Processing image: 35TH ST SE FROM POPE ST SE TO CARPENTER ST SE.jpg
Processing image: 35TH ST NW FROM WINFIELD LN NW TO RESERVOIR RD NW.jpg
Processing image: 42ND ST NE FROM BROOKS ST NE TO CLAY ST NE.jpg
Processing image: 35TH ST NW FROM WINFIELD LN NW TO RESERVOIR RD NW (2).jpg
Processing image: 35TH ST NW FROM VOLTA PL NW TO Q ST NW (1).jpg
Processing image: 35TH ST NW FROM WINFIELD LN NW TO RESERVOIR RD NW (1).jpg
Processing image: 32ND ST SE FROM D ST SE TO ELY PL SE (2).jpg
Processing image: 35TH ST NW FROM WINFIELD LN NW TO RESERVOIR RD NW (30.jpg
Processing image: 30TH ST NE FROM ADAMS ST NE TO CHANNING ST NE (2).jpg
Processing image: 36TH ST NW FROM APPLETON ST NW TO BRANDYWINE ST NW (2).jpg
Processing image: 23343 pic 405-435A Superpave HMA SP3 2023-05-31 #02.jpg


Epochs: The training was performed for 10 epochs.
Loss and Accuracy: The training loss decreased from 0.5190 to 0.1615, and the training accuracy remained constant at approximately 89.06%. Similarly, the validation loss decreased from 0.2134 to 0.1536, and the validation accuracy remained constant at approximately 88.63%.
Evaluation: After training, the model was evaluated on the validation data, resulting in a validation loss of approximately 0.1536 and a validation accuracy of approximately 88.63%.